In [8]:
import os
import tensorflow as tf
os.environ['CUDA_VISIBLE_DEVICES'] = "3"

import numpy as np
import torch
import torch.utils.data as data

import json
import functools
from PIL import Image
import argparse
import random
try:
    import accimage
except ImportError:
    accimage = None

def pil_loader(path):
    with open(path, 'rb') as f:
        with Image.open(f) as img:
            return img.convert('RGB')
        
def get_default_image_loader():
    from torchvision import get_image_backend
    if get_image_backend() == 'accimage':
        return accimage_loader
    else:
        return pil_loader

def accimage_loader(path):
    try:
        return accimage.Image(path)
    except IOError:
        # Potentially a decoding problem, fall back to PIL.Image
        return pil_loader(path)

    
def get_default_video_loader():
    image_loader = get_default_image_loader()
    return functools.partial(video_loader, image_loader=image_loader)

def load_annotation_data(path):
    with open(path, 'r') as data_file:
        return json.load(data_file)
    
def load_value_file(file_path):
    with open(file_path, 'r') as input_file:
        value = float(input_file.read().rstrip('\n\r'))

    return value

def video_loader(video_dir_path, frame_indices, image_loader):
    video = []
    for i in frame_indices:
        image_path = os.path.join(video_dir_path, 'image_{:05d}.jpg'.format(i))
        if os.path.exists(image_path):
            video.append(image_loader(image_path))
        else:
            return video

    return video

def get_video_names_and_annotations(data, subset):
    video_names = []
    annotations = []

    for key, value in data['database'].items():
        this_subset = value['subset']
        if this_subset == subset:
            label = value['annotations']['label']
            video_names.append('{}/{}'.format(label, key))
            annotations.append(value['annotations'])

    return video_names, annotations

def get_class_labels(data):
    class_labels_map = {}
    index = 0
    for class_label in data['labels']:
        class_labels_map[class_label] = index
        index += 1
    return class_labels_map
    
def make_dataset(root_path, annotation_path, subset, n_samples_for_each_video, sample_duration):
    data = load_annotation_data(annotation_path)
    video_names, annotations = get_video_names_and_annotations(data, subset)
    class_to_idx = get_class_labels(data)
    idx_to_class = {}
    for name, label in class_to_idx.items():
        idx_to_class[label] = name
        
    dataset = []
    for i in range(len(video_names)):
        if i % 1000 == 0:
            print('dataset loading [{}/{}]'.format(i, len(video_names)))

        video_path = os.path.join(root_path, video_names[i])
        
        if not os.path.exists(video_path):
            continue

        n_frames_file_path = os.path.join(video_path, 'n_frames')
        n_frames = int(load_value_file(n_frames_file_path))
        if n_frames <= 0:
            continue

        begin_t = 1
        end_t = n_frames
        sample = {
            'video': video_path,
            'segment': [begin_t, end_t],
            'n_frames': n_frames,
            'video_id': video_names[i].split('/')[1]
        }
        if len(annotations) != 0:
            sample['label'] = class_to_idx[annotations[i]['label']]
        else:
            sample['label'] = -1

        if n_samples_for_each_video == 1:
            sample['frame_indices'] = list(range(1, n_frames + 1))
            dataset.append(sample)
        else:
            if n_samples_for_each_video > 1:
                step = max(1,
                           math.ceil((n_frames - 1 - sample_duration) /
                                     (n_samples_for_each_video - 1)))
            else:
                step = sample_duration
            for j in range(1, n_frames, step):
                sample_j = copy.deepcopy(sample)
                sample_j['frame_indices'] = list(
                    range(j, min(n_frames + 1, j + sample_duration)))
                dataset.append(sample_j)

    return dataset, idx_to_class

class UCF101(data.Dataset):
    def __init__(self, root_path, annotation_path, subset, spatial_transform, temporal_transform, target_transform, sample_duration = 16, get_loader=get_default_video_loader,
                 n_samples_for_each_video=1):
        self.data, self.class_names = make_dataset(root_path, annotation_path, subset, n_samples_for_each_video, sample_duration)
        self.spatial_transform = spatial_transform 
        self.temporal_transform = temporal_transform 
        self.target_transform = target_transform
        self.loader = get_loader()
    
    def __getitem__(self, index):
        path = self.data[index]['video']

        frame_indices = self.data[index]['frame_indices']
        if self.temporal_transform is not None:
            frame_indices = self.temporal_transform(frame_indices)
        clip = self.loader(path, frame_indices)
        if self.spatial_transform is not None:
            self.spatial_transform.randomize_parameters()
            clip = [self.spatial_transform(img) for img in clip]
        clip = torch.stack(clip, 0).permute(1, 0, 2, 3)

        target = self.data[index]
        if self.target_transform is not None:
            target = self.target_transform(target)

        return clip, target
    
    def __len__(self):
        return len(self.data)
    
class Compose(object):
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, img):
        for t in self.transforms:
            img = t(img)
        return img

    def randomize_parameters(self):
        for t in self.transforms:
            t.randomize_parameters()
            
class ClassLabel(object):
    def __call__(self, target):
        return target['label']
    
class TemporalRandomCrop(object):
    def __init__(self, size):
        self.size = size

    def __call__(self, frame_indices):
        rand_end = max(0, len(frame_indices) - self.size - 1)
        begin_index = random.randint(0, rand_end)
        end_index = min(begin_index + self.size, len(frame_indices))

        out = frame_indices[begin_index:end_index]

        for index in out:
            if len(out) >= self.size:
                break
            out.append(index)

        return out

class RandomHorizontalFlip(object):
    def __call__(self, img):
        if self.p < 0.5:
            return img.transpose(Image.FLIP_LEFT_RIGHT)
        return img

    def randomize_parameters(self):
        self.p = random.random()

class ToTensor(object):
    def __init__(self, norm_value=255):
        self.norm_value = norm_value

    def __call__(self, pic):
        if isinstance(pic, np.ndarray):
            # handle numpy array
            img = torch.from_numpy(pic.transpose((2, 0, 1)))
            # backward compatibility
            return img.float().div(self.norm_value)

        if accimage is not None and isinstance(pic, accimage.Image):
            nppic = np.zeros(
                [pic.channels, pic.height, pic.width], dtype=np.float32)
            pic.copyto(nppic)
            return torch.from_numpy(nppic)

        # handle PIL Image
        if pic.mode == 'I':
            img = torch.from_numpy(np.array(pic, np.int32, copy=False))
        elif pic.mode == 'I;16':
            img = torch.from_numpy(np.array(pic, np.int16, copy=False))
        else:
            img = torch.ByteTensor(torch.ByteStorage.from_buffer(pic.tobytes()))
        # PIL image mode: 1, L, P, I, F, RGB, YCbCr, RGBA, CMYK
        if pic.mode == 'YCbCr':
            nchannel = 3
        elif pic.mode == 'I;16':
            nchannel = 1
        else:
            nchannel = len(pic.mode)
        img = img.view(pic.size[1], pic.size[0], nchannel)
        # put it from HWC to CHW format
        # yikes, this transpose takes 80% of the loading time/CPU
        img = img.transpose(0, 1).transpose(0, 2).contiguous()
        if isinstance(img, torch.ByteTensor):
            return img.float().div(self.norm_value)
        else:
            return img

    def randomize_parameters(self):
        pass

class Normalize(object):
    def __init__(self, mean, std):
        self.mean = mean
        self.std = std

    def __call__(self, tensor):
        # TODO: make efficient
        for t, m, s in zip(tensor, self.mean, self.std):
            t.sub_(m).div_(s)
        return tensor

    def randomize_parameters(self):
        pass

class MultiScaleCornerCrop(object):
    def __init__(self,
                 scales,
                 size,
                 interpolation=Image.BILINEAR,
                 crop_positions=['c', 'tl', 'tr', 'bl', 'br']):
        self.scales = scales
        self.size = size
        self.interpolation = interpolation

        self.crop_positions = crop_positions

    def __call__(self, img):
        print(img.size)
        min_length = min(img.size[0], img.size[1])
        crop_size = int(min_length * self.scale)

        image_width = img.size[0]
        image_height = img.size[1]

        if self.crop_position == 'c':
            center_x = image_width // 2
            center_y = image_height // 2
            box_half = crop_size // 2
            x1 = center_x - box_half
            y1 = center_y - box_half
            x2 = center_x + box_half
            y2 = center_y + box_half
        elif self.crop_position == 'tl':
            x1 = 0
            y1 = 0
            x2 = crop_size
            y2 = crop_size
        elif self.crop_position == 'tr':
            x1 = image_width - crop_size
            y1 = 0
            x2 = image_width
            y2 = crop_size
        elif self.crop_position == 'bl':
            x1 = 0
            y1 = image_height - crop_size
            x2 = crop_size
            y2 = image_height
        elif self.crop_position == 'br':
            x1 = image_width - crop_size
            y1 = image_height - crop_size
            x2 = image_width
            y2 = image_height

        img = img.crop((x1, y1, x2, y2))

        return img.resize((self.size, self.size), self.interpolation)

    def randomize_parameters(self):
        self.scale = self.scales[random.randint(0, len(self.scales) - 1)]
        self.crop_position = self.crop_positions[random.randint(
            0,
            len(self.crop_positions) - 1)]

class Scale(object):
    def __init__(self, size, interpolation=Image.BILINEAR):
        assert isinstance(size,
                          int) or (isinstance(size, collections.Iterable) and
                                   len(size) == 2)
        self.size = size
        self.interpolation = interpolation

    def __call__(self, img):
        if isinstance(self.size, int):
            w, h = img.size
            if (w <= h and w == self.size) or (h <= w and h == self.size):
                return img
            if w < h:
                ow = self.size
                oh = int(self.size * h / w)
                return img.resize((ow, oh), self.interpolation)
            else:
                oh = self.size
                ow = int(self.size * w / h)
                return img.resize((ow, oh), self.interpolation)
        else:
            return img.resize(self.size, self.interpolation)

    def randomize_parameters(self):
        pass

class CornerCrop(object):

    def __init__(self, size, crop_position=None):
        self.size = size
        if crop_position is None:
            self.randomize = True
        else:
            self.randomize = False
        self.crop_position = crop_position
        self.crop_positions = ['c', 'tl', 'tr', 'bl', 'br']

    def __call__(self, img):
        image_width = img.size[0]
        image_height = img.size[1]

        if self.crop_position == 'c':
            th, tw = (self.size, self.size)
            x1 = int(round((image_width - tw) / 2.))
            y1 = int(round((image_height - th) / 2.))
            x2 = x1 + tw
            y2 = y1 + th
        elif self.crop_position == 'tl':
            x1 = 0
            y1 = 0
            x2 = self.size
            y2 = self.size
        elif self.crop_position == 'tr':
            x1 = image_width - self.size
            y1 = 0
            x2 = image_width
            y2 = self.size
        elif self.crop_position == 'bl':
            x1 = 0
            y1 = image_height - self.size
            x2 = self.size
            y2 = image_height
        elif self.crop_position == 'br':
            x1 = image_width - self.size
            y1 = image_height - self.size
            x2 = image_width
            y2 = image_height

        img = img.crop((x1, y1, x2, y2))

        return img

    def randomize_parameters(self):
        if self.randomize:
            self.crop_position = self.crop_positions[random.randint(
                0,
                len(self.crop_positions) - 1)]

class LoopPadding(object):

    def __init__(self, size):
        self.size = size

    def __call__(self, frame_indices):
        out = frame_indices

        for index in out:
            if len(out) >= self.size:
                break
            out.append(index)

        return out
    
class VideoID(object):
    def __call__(self, target):
        return target['video_id']
    
def get_training_set(video_path, annotation_path, spatial_transform, temporal_transform, target_transform):
    training_data = UCF101(
            video_path,
            annotation_path,
            'training',
            spatial_transform, 
            temporal_transform, 
            target_transform)
    return training_data

def get_testing_set(video_path, annotation_path, spatial_transform, temporal_transform, target_transform):    
    test_data = UCF101(
            video_path,
            annotation_path,
            'validation',
            spatial_transform,
            temporal_transform,
            target_transform)

    return test_data

def get_mean(norm_value=255):
    return [114.7748 / norm_value, 107.7354 / norm_value, 99.4750 / norm_value]

In [9]:
from torchvision import transforms

scales = [1]
for i in range(1, 3):
    scales.append(scales[-1] * 0.84089641525)
    
mean = get_mean(1)
norm_method = Normalize(mean, [1, 1, 1])

spatial_transform = Compose([
            MultiScaleCornerCrop(scales, 56),
            RandomHorizontalFlip(),
            ToTensor(1), norm_method
        ])
temporal_transform = TemporalRandomCrop(16)
target_transform = ClassLabel()

training_data = get_training_set('./data/ucf101/jpg', './data/ucf101_01.json', spatial_transform, temporal_transform, target_transform)
testing_data = get_testing_set('./data/ucf101/jpg', './data/ucf101_01.json', spatial_transform, temporal_transform, target_transform)

train_loader = torch.utils.data.DataLoader(
            training_data,
            batch_size=64,
            shuffle=True,
            num_workers=4,
            pin_memory=True)

test_loader = torch.utils.data.DataLoader(
            testing_data,
            batch_size=64,
            shuffle=True,
            num_workers=4,
            pin_memory=True)

dataset loading [0/9537]
dataset loading [1000/9537]
dataset loading [2000/9537]
dataset loading [3000/9537]
dataset loading [4000/9537]
dataset loading [5000/9537]
dataset loading [6000/9537]
dataset loading [7000/9537]
dataset loading [8000/9537]
dataset loading [9000/9537]
dataset loading [0/3783]
dataset loading [1000/3783]
dataset loading [2000/3783]
dataset loading [3000/3783]


In [6]:
for i, (inputs, targets) in enumerate(train_loader):
    print(inputs.shape)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)


...

Process Process-2:
  File "/usr/lib/python3/dist-packages/ipykernel/iostream.py", line 364, in _flush
    parent=self.parent_header, ident=self.topic)
ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.

Process Process-4:
Process Process-3:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
Traceback (most recent call last):
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()


Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py", line 2882, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-6-feea7426e491>", line 1, in <module>
    for i, (inputs, targets) in enumerate(train_loader):
  File "/usr/lib/python3/dist-packages/torch/utils/data/dataloader.py", line 330, in __next__
    idx, batch = self._get_batch()
  File "/usr/lib/python3/dist-packages/torch/utils/data/dataloader.py", line 309, in _get_batch
    return self.data_queue.get()
  File "/usr/lib/python3.6/queue.py", line 164, in get
    self.not_empty.wait()
  File "/usr/lib/python3.6/threading.py", line 295, in wait
    waiter.acquire()
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/lib/python3/dist-packages/IPython/core/interactiveshell.py", line 1823, in showtraceback
    stb = value._render_traceback_()
AttributeError

  File "/usr/lib/python3/dist-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])


KeyboardInterrupt: 

  File "/usr/lib/python3/dist-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/lib/python3.6/multiprocessing/process.py", line 258, in _bootstrap
    self.run()
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3/dist-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/lib/python3/dist-packages/torch/utils/data/dataloader.py", line 106, in <listcomp>
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "/usr/lib/python3.6/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3/dist-packages/torch/utils/data/dataloader.py", line 106, in _worker_loop
    samples = collate_fn([dataset[i] for i in batch_indices])
  File "<ipython-input-3-83e8de64

In [10]:
for i, (inputs, targets) in enumerate(test_loader):
    print(inputs.shape)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)


...

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)

(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)
(320, 240)